In [7]:
import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np

In [8]:
from bokeh.io import output_notebook, show, save
output_notebook()

Loading BokehJS ...

In [9]:
dataframe_all_transfers = pd.read_csv('./data/network_all_transfers_202101_202107.csv')
dataframe_incoming_transfers = pd.read_csv('./data/network_incoming_transfers_202101_202107.csv')
dataframe_outgoing_transfers = pd.read_csv('./data/network_outgoing_transfers_202101_202107.csv')

In [10]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [11]:
def visualize_network(dataframe, show_labels=False):
    network = networkx.from_pandas_edgelist(dataframe, 'shipper_name', 'recipient_name', 'count')
    # Calculate degree for each node and add as node attribute
    degrees = dict(networkx.degree(network))
    networkx.set_node_attributes(network, name='degree', values=degrees)
    # Slightly adjust degree so that the nodes with very small degrees are still visible
    number_to_adjust_by = 5
    adjusted_node_size = dict([(node, min(250, degree + number_to_adjust_by)) for node, degree in networkx.degree(network)])
    networkx.set_node_attributes(network, name='adjusted_node_size', values=adjusted_node_size)

    #Choose colors for node and edge highlighting
    node_highlight_color = 'white'
    edge_highlight_color = 'black'

    #Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
    size_by_this_attribute = 'adjusted_node_size'
    color_by_this_attribute = 'adjusted_node_size'

    #Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
    color_palette = Blues8

    #Choose a title!
    title = 'Cannabis Industry Network'

    #Establish which categories will appear when hovering over each node
    HOVER_TOOLTIPS = [
        ("License", "@index"),
        ("Degree", "@degree")
    ]

    #Create a plot — set dimensions, toolbar, and title
    plot = figure(
        tooltips = HOVER_TOOLTIPS,
        tools="pan,wheel_zoom,save,reset",
        active_scroll='wheel_zoom',
        x_range=Range1d(-100.1, 100.1),
        y_range=Range1d(-100.1, 100.1),
        title=title
    )

    #Create a network graph object
    # https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
    network_graph = from_networkx(network, networkx.spring_layout, scale=100, center=(0, 0))

    #Set node sizes and colors according to node degree (color as spectrum of color palette)
    minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
    network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))
    #Set node highlight colors
    network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
    network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

    #Set edge opacity and width
    network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)
    #Set edge highlight colors
    network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
    network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

    #Highlight nodes and edges
    network_graph.selection_policy = NodesAndLinkedEdges()
    network_graph.inspection_policy = NodesAndLinkedEdges()

    plot.renderers.append(network_graph)
    
    if show_labels:
        #Add Labels
        x, y = zip(*network_graph.layout_provider.graph_layout.values())
        node_labels = list(network.nodes())
        source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
        labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
        plot.renderers.append(labels)

    show(plot)

In [12]:
visualize_network(dataframe_all_transfers)

In [13]:
visualize_network(dataframe_incoming_transfers)

In [14]:
visualize_network(dataframe_outgoing_transfers)